Notebook for listhost signup -- have to sign into listhost, then interact with page

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import logging
import time
import json

In [20]:
with open('../login.json', 'r') as file:
    login = json.load(file)

# Access the credentials
email = login['email']
password = login['password']

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='listhost_login.log'
)

In [16]:
class ListhostLogin:
    def __init__(self, email: str, password: str):
        self.email = email
        self.password = password
        self.driver = webdriver.Chrome()
        self.wait = WebDriverWait(self.driver, 10)
        
    def login(self) -> bool:
        """Handle the complete login process"""
        try:
            # 1. Navigate to the main page
            logging.info("Navigating to main page...")
            self.driver.get("https://lists.uchicago.edu/web")
            time.sleep(2)  # Wait for page to fully load
            
            # 2. Click the initial login button
            logging.info("Clicking initial login button...")
            login_button = self.wait.until(
                EC.element_to_be_clickable((By.NAME, "action_login"))
            )
            login_button.click()
            time.sleep(1.5)  # Wait for transition
            
            # Wait for page transition (checking for email field presence)
            email_field = self.wait.until(
                EC.presence_of_element_located((By.ID, "email_login"))
            )
            
            # 3. Fill in credentials
            logging.info("Filling credentials...")
            email_field.send_keys(self.email)
            time.sleep(0.5)  # Small wait between fields
            
            password_field = self.driver.find_element(By.ID, "passwd")
            password_field.send_keys(self.password)
            time.sleep(0.5)  # Wait before clicking submit
            
            # 4. Click the login submit button
            logging.info("Submitting login form...")
            submit_button = self.driver.find_element(
                By.CSS_SELECTOR, 
                "button[class='heavyWork'][type='submit'][name='action_login']"
            )
            submit_button.click()
            time.sleep(2)  # Wait for login processing
            
            # 5. Wait for successful login (checking for email element)
            self.wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, f'li[aria-label*="{self.email}"]')
                )
            )
            
            logging.info("Successfully logged in!")
            time.sleep(1)  # Final wait to ensure everything is loaded
            return True
            
        except TimeoutException as e:
            logging.error(f"Timeout while waiting for element: {str(e)}")
            return False
        except Exception as e:
            logging.error(f"Error during login process: {str(e)}")
            return False
    
    def is_logged_in(self) -> bool:
        """Check if we're currently logged in"""
        try:
            email_element = self.driver.find_element(
                By.CSS_SELECTOR, 
                f'li[aria-label*="{self.email}"]'
            )
            return email_element is not None
        except:
            return False
    
    def close(self):
        """Close the browser"""
        self.driver.quit()


In [12]:

# Usage example
def test_login(email: str, password: str):
    login_handler = ListhostLogin(email, password)
    try:
        success = login_handler.login()
        if success:
            print("Login successful!")
            # Here we could continue with list subscription logic
        else:
            print("Login failed!")
    finally:
        login_handler.close()

In [21]:
test_login(email, password)

Login successful!
